In [1]:
import pandas as pd
import lpips
import cv2
import torch
import numpy as np
import os
from tqdm import tqdm
from scipy.stats import pearsonr, spearmanr
from DISTS_pytorch import DISTS
import pyiqa

In [2]:
dataset = pd.read_csv("koniq10k_scores_and_distributions.csv")
max_mos = dataset["MOS"].values.max()
min_mos = dataset["MOS"].values.min()

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
loss_fn_alex = lpips.LPIPS(net='alex')
D = DISTS().to(device)
nima = pyiqa.create_metric('nima', device=device)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


c:\Users\nikis\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\nikis\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: c:\Users\nikis\anaconda3\Lib\site-packages\lpips\weights\v0.1\alex.pth


c:\Users\nikis\anaconda3\Lib\site-packages\lpips\lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, map_location='cpu'),

Loading pretrained model NIMA from C:\Users\nikis\.cache\torch\hub\checkpoints\NIMA_InceptionV2_ava-b0c77c00.pth


c:\Users\nikis\anaconda3\Lib\site-packages\pyiqa\archs\arch_util.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=torch

In [5]:
esrgan_lpips_scores = {}
esrgan_lpips_scores["image_name"] = []
esrgan_lpips_scores["MOS"] = []

Чем меньше значение LPIPS, тем ближе изображения.

Требуется сделать преобразование в MOS, которое есть в KonIQ-10k. Из вывода LPIPS кажется, что значения варьируются от 0 до 1 (также в статье присутствует сигмоида, которая выдаёт значения от 0 до 1). Следовательно, преобразование расчитывал по следующей формуле:

$$ (1 - d) = \frac{d_{\text{MOS}} - min}{max - min}, $$
$$ d_{\text{MOS}} = (1 - d) \cdot (max - min) + min $$

In [6]:
count = 0
images = os.listdir("1024x768")
for image_name in tqdm(images):
    esrgan_lpips_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"1024x768/{image_name}")
    img1 = cv2.imread(f"ESRGAN/results/{image_name[:-4]}_rlt.png")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = (img0 / 127.5) - 1
    img1 = (img1 / 127.5) - 1

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0)
    img1 = torch.tensor(img1)

    d = loss_fn_alex(img0, img1)
    esrgan_lpips_scores["MOS"].append(float((d * (max_mos - min_mos) + min_mos)[0, 0, 0, 0]))
    df = pd.DataFrame(esrgan_lpips_scores)
    df.to_csv("ESRGAN_LPIPS_scores.csv")
    count += 1
    if count == 3746:
        break

 36%|███▌      | 3745/10373 [27:51<49:18,  2.24it/s]  


In [7]:
real_esrgan_lpips_scores = {}
real_esrgan_lpips_scores["image_name"] = []
real_esrgan_lpips_scores["MOS"] = []

In [8]:
count = 0
images = os.listdir("1024x768")
for image_name in tqdm(images):
    real_esrgan_lpips_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"1024x768/{image_name}")
    img1 = cv2.imread(f"Real-ESRGAN/results/{image_name[:-4]}_out.jpg")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = (img0 / 127.5) - 1
    img1 = (img1 / 127.5) - 1

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0)
    img1 = torch.tensor(img1)

    d = loss_fn_alex(img0, img1)
    real_esrgan_lpips_scores["MOS"].append(float((d * (max_mos - min_mos) + min_mos)[0, 0, 0, 0]))
    df = pd.DataFrame(real_esrgan_lpips_scores)
    df.to_csv("Real-ESRGAN_LPIPS_scores.csv")
    count += 1
    if count == 3746:
        break

 36%|███▌      | 3745/10373 [27:16<48:16,  2.29it/s]  


In [9]:
esrgan_dists_scores = {}
esrgan_dists_scores["image_name"] = []
esrgan_dists_scores["MOS"] = []

In [10]:
count = 0
images = os.listdir("1024x768")
for image_name in tqdm(images):
    esrgan_dists_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"1024x768/{image_name}")
    img1 = cv2.imread(f"ESRGAN/results/{image_name[:-4]}_rlt.png")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = img0 / 255
    img1 = img1 / 255

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0).to(device)
    img1 = torch.tensor(img1).to(device)

    d = D(img0, img1)
    del img0
    del img1
    esrgan_dists_scores["MOS"].append(float((d * (max_mos - min_mos) + min_mos)))
    df = pd.DataFrame(esrgan_dists_scores)
    df.to_csv("ESRGAN_DISTS_scores.csv")
    count += 1
    if count == 3746:
        break

 36%|███▌      | 3745/10373 [17:54<31:42,  3.48it/s]


In [11]:
real_esrgan_dists_scores = {}
real_esrgan_dists_scores["image_name"] = []
real_esrgan_dists_scores["MOS"] = []

In [12]:
count = 0
images = os.listdir("1024x768")
for image_name in tqdm(images):
    real_esrgan_dists_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"1024x768/{image_name}")
    img1 = cv2.imread(f"Real-ESRGAN/results/{image_name[:-4]}_out.jpg")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = img0 / 255
    img1 = img1 / 255

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0).to(device)
    img1 = torch.tensor(img1).to(device)

    d = D(img0, img1)
    del img0
    del img1
    real_esrgan_dists_scores["MOS"].append(float((d * (max_mos - min_mos) + min_mos)))
    df = pd.DataFrame(real_esrgan_dists_scores)
    df.to_csv("Real-ESRGAN_DISTS_scores.csv")
    count += 1
    if count == 3746:
        break

 36%|███▌      | 3745/10373 [17:33<31:04,  3.55it/s]


In [13]:
esrgan_nima_scores = {}
esrgan_nima_scores["image_name"] = []
esrgan_nima_scores["MOS"] = []

In [14]:
count = 0
images = os.listdir("1024x768")
for image_name in tqdm(images):
    esrgan_nima_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"1024x768/{image_name}")
    img1 = cv2.imread(f"ESRGAN/results/{image_name[:-4]}_rlt.png")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = img0 / 255
    img1 = img1 / 255

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0)
    img1 = torch.tensor(img1)

    d = nima(img0, img1)
    esrgan_nima_scores["MOS"].append(float(((d / 10) * (max_mos - min_mos) + min_mos)))
    df = pd.DataFrame(esrgan_nima_scores)
    df.to_csv("ESRGAN_NIMA_scores.csv")
    count += 1
    if count == 3746:
        break

 36%|███▌      | 3745/10373 [09:35<16:58,  6.51it/s]


In [15]:
real_esrgan_nima_scores = {}
real_esrgan_nima_scores["image_name"] = []
real_esrgan_nima_scores["MOS"] = []

In [16]:
count = 0
images = os.listdir("1024x768")
for image_name in tqdm(images):
    real_esrgan_nima_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"1024x768/{image_name}")
    img1 = cv2.imread(f"Real-ESRGAN/results/{image_name[:-4]}_out.jpg")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = img0 / 255
    img1 = img1 / 255

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0)
    img1 = torch.tensor(img1)

    d = nima(img0, img1)
    real_esrgan_nima_scores["MOS"].append(float(((d / 10) * (max_mos - min_mos) + min_mos)))
    df = pd.DataFrame(real_esrgan_nima_scores)
    df.to_csv("Real-ESRGAN_NIMA_scores.csv")
    count += 1
    if count == 3746:
        break

 36%|███▌      | 3745/10373 [09:03<16:01,  6.90it/s]


In [23]:
esrgan_ahiq_scores = {}
esrgan_ahiq_scores["image_name"] = []
esrgan_ahiq_scores["MOS"] = []

In [ ]:
ahiq = pyiqa.create_metric('ahiq', device=device)

count = 0
images = os.listdir("1024x768")
for image_name in tqdm(images):
    esrgan_ahiq_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"1024x768/{image_name}")
    img1 = cv2.imread(f"ESRGAN/results/{image_name[:-4]}_rlt.png")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = img0 / 255
    img1 = img1 / 255

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0)
    img1 = torch.tensor(img1)

    d = ahiq(img0, img1)
    esrgan_ahiq_scores["MOS"].append(float((d * (max_mos - min_mos) + min_mos)))
    df = pd.DataFrame(esrgan_ahiq_scores)
    df.to_csv("ESRGAN_AHIQ_scores.csv")
    count += 1
    if count == 3746:
        break

In [ ]:
real_esrgan_ahiq_scores = {}
real_esrgan_ahiq_scores["image_name"] = []
real_esrgan_ahiq_scores["MOS"] = []

In [ ]:
count = 0
images = os.listdir("1024x768")
for image_name in tqdm(images):
    real_esrgan_ahiq_scores["image_name"].append(image_name)
    img0 = cv2.imread(f"1024x768/{image_name}")
    img1 = cv2.imread(f"Real-ESRGAN/results/{image_name[:-4]}_out.jpg")

    img0 = img0.reshape(1, 3, 768, 1024)
    img1 = img1.reshape(1, 3, 768, 1024)

    img0 = img0 / 255
    img1 = img1 / 255

    img0 = img0.astype(np.float32)
    img1 = img1.astype(np.float32)

    img0 = torch.tensor(img0)
    img1 = torch.tensor(img1)

    d = ahiq(img0, img1)
    real_esrgan_ahiq_scores["MOS"].append(float((d * (max_mos - min_mos) + min_mos)))
    df = pd.DataFrame(real_esrgan_ahiq_scores)
    df.to_csv("Real-ESRGAN_AHIQ_scores.csv")
    count += 1
    if count == 3746:
        break

In [19]:
df_ESRGAN_lpips = pd.read_csv("ESRGAN_LPIPS_scores.csv")
df_Real_ESRGAN_lpips = pd.read_csv("Real-ESRGAN_LPIPS_scores.csv")
df_ESRGAN_dists = pd.read_csv("ESRGAN_DISTS_scores.csv")
df_Real_ESRGAN_dists = pd.read_csv("Real-ESRGAN_DISTS_scores.csv")
df_ESRGAN_nima = pd.read_csv("ESRGAN_NIMA_scores.csv")
df_Real_ESRGAN_nima = pd.read_csv("Real-ESRGAN_NIMA_scores.csv")

correlations = {}

pearson_esrgan_lpips = round(pearsonr(dataset["MOS"].values[:3746], df_ESRGAN_lpips["MOS"].values)[0], 3)
pearson_real_esrgan_lpips = round(pearsonr(dataset["MOS"].values[:3746], df_Real_ESRGAN_lpips["MOS"].values)[0], 3)
pearson_esrgan_dists = round(pearsonr(dataset["MOS"].values[:3746], df_ESRGAN_dists["MOS"].values)[0], 3)
pearson_real_esrgan_dists = round(pearsonr(dataset["MOS"].values[:3746], df_Real_ESRGAN_dists["MOS"].values)[0], 3)
pearson_esrgan_nima = round(pearsonr(dataset["MOS"].values[:3746], df_ESRGAN_nima["MOS"].values)[0], 3)
pearson_real_esrgan_nima = round(pearsonr(dataset["MOS"].values[:3746], df_Real_ESRGAN_nima["MOS"].values)[0], 3)

spearman_esrgan_lpips = round(spearmanr(dataset["MOS"].values[:3746], df_ESRGAN_lpips["MOS"].values)[0], 3)
spearman_real_esrgan_lpips = round(spearmanr(dataset["MOS"].values[:3746], df_Real_ESRGAN_lpips["MOS"].values)[0], 3)
spearman_esrgan_dists = round(spearmanr(dataset["MOS"].values[:3746], df_ESRGAN_dists["MOS"].values)[0], 3)
spearman_real_esrgan_dists = round(spearmanr(dataset["MOS"].values[:3746], df_Real_ESRGAN_dists["MOS"].values)[0], 3)
spearman_esrgan_nima = round(spearmanr(dataset["MOS"].values[:3746], df_ESRGAN_nima["MOS"].values)[0], 3)
spearman_real_esrgan_nima = round(spearmanr(dataset["MOS"].values[:3746], df_Real_ESRGAN_nima["MOS"].values)[0], 3)

cosine_esrgan_lpips = round(np.dot(dataset["MOS"].values[:3746], df_ESRGAN_lpips["MOS"].values) / (np.linalg.norm(dataset["MOS"].values[:3746]) * np.linalg.norm(df_ESRGAN_lpips["MOS"].values)), 3)
cosine_real_esrgan_lpips = round(np.dot(dataset["MOS"].values[:3746], df_Real_ESRGAN_lpips["MOS"].values) / (np.linalg.norm(dataset["MOS"].values[:3746]) * np.linalg.norm(df_Real_ESRGAN_lpips["MOS"].values)), 3)
cosine_esrgan_dists = round(np.dot(dataset["MOS"].values[:3746], df_ESRGAN_dists["MOS"].values) / (np.linalg.norm(dataset["MOS"].values[:3746]) * np.linalg.norm(df_ESRGAN_dists["MOS"].values)))
cosine_real_esrgan_dists = round(np.dot(dataset["MOS"].values[:3746], df_Real_ESRGAN_dists["MOS"].values) / (np.linalg.norm(dataset["MOS"].values[:3746]) * np.linalg.norm(df_Real_ESRGAN_dists["MOS"].values)), 3)
cosine_esrgan_nima = round(np.dot(dataset["MOS"].values[:3746], df_ESRGAN_nima["MOS"].values) / (np.linalg.norm(dataset["MOS"].values[:3746]) * np.linalg.norm(df_ESRGAN_nima["MOS"].values)))
cosine_real_esrgan_nima = round(np.dot(dataset["MOS"].values[:3746], df_Real_ESRGAN_nima["MOS"].values) / (np.linalg.norm(dataset["MOS"].values[:3746]) * np.linalg.norm(df_Real_ESRGAN_nima["MOS"].values)), 3)

correlations = {
    "LIPIPS": [
        f"{pearson_esrgan_lpips}/{spearman_esrgan_lpips}/{cosine_esrgan_lpips}",
        f"{pearson_real_esrgan_lpips}/{spearman_real_esrgan_lpips}/{cosine_real_esrgan_lpips}"
    ],
    "DISTS": [
        f"{pearson_esrgan_dists}/{spearman_esrgan_dists}/{cosine_esrgan_dists}",
        f"{pearson_real_esrgan_dists}/{spearman_real_esrgan_dists}/{cosine_real_esrgan_dists}"
    ],
    "NIMA": [
        f"{pearson_esrgan_nima}/{spearman_esrgan_nima}/{cosine_esrgan_nima}",
        f"{pearson_real_esrgan_nima}/{spearman_real_esrgan_nima}/{cosine_real_esrgan_nima}"
    ]
}

correlations_df = pd.DataFrame(correlations, index=["ESRGAN", "Real-ESRGAN"])
correlations_df.to_csv("correlations.csv")

In [20]:
correlations_df

,LIPIPS,DISTS,NIMA
ESRGAN,0.024/0.03/0.967,0.033/0.039/1,0.045/0.035/1
Real-ESRGAN,0.01/0.022/0.976,0.018/0.025/0.977,0.045/0.035/0.981
